<a href="https://colab.research.google.com/github/RickyF404/Tesi/blob/main/Anomaly_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import gdown

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer

In [2]:
url_no_temp = "https://drive.google.com/uc?id=1jerpsOqjogEnnriBkHOrDDEeh1wJry_o"
output_no_temp = "data_no_temperature.csv"
gdown.download(url_no_temp, output_no_temp)

url_temp = " https://drive.google.com/uc?id=1RofjUHZS_UAnbF6Xe74jTp9z4Mmk7swH"
output_temp = "data_temperature.csv"
gdown.download(url_temp, output_temp)


url_anomaly = "https://drive.google.com/uc?id=1onDmjfJio6nVRaaQSOgd6wW_XyuyyCra"
output_anomaly = "data_anomaly.csv"
gdown.download(url_anomaly, output_anomaly)

df_anomaly = pd.read_csv("/content/data_anomaly.csv", encoding = "utf-8")

mapping = {
    "F1": "Mode 1",
    "F2": "Mode 2",
    "F3": "Mode 3",
    "F4": "Mode 4",
    "F5": "Mode 5",
    "F6": "Mode 6",
    "Temperature": "temperatura"
}

df_anomaly = df_anomaly.rename(columns=mapping)

Downloading...
From: https://drive.google.com/uc?id=1jerpsOqjogEnnriBkHOrDDEeh1wJry_o
To: /content/data_no_temperature.csv
100%|██████████| 1.44M/1.44M [00:00<00:00, 15.1MB/s]
Downloading...
From:  https://drive.google.com/uc?id=1RofjUHZS_UAnbF6Xe74jTp9z4Mmk7swH
To: /content/data_temperature.csv
100%|██████████| 1.66M/1.66M [00:00<00:00, 80.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1onDmjfJio6nVRaaQSOgd6wW_XyuyyCra
To: /content/data_anomaly.csv
100%|██████████| 990k/990k [00:00<00:00, 35.1MB/s]


# CVAE beta = 1

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="keras")

df_temp = pd.read_csv("/content/data_temperature.csv", encoding = "utf-8")
df_temp["timestamp"] = pd.to_datetime(df_temp["timestamp"])
df_temp["timestamp"] = df_temp["timestamp"].dt.tz_localize(None)
df_temp = df_temp.sort_values("timestamp")

end = "2025-01-01"
new_df = df_temp[(df_temp["timestamp"] < end)]

class Sampling(Layer):
  def call(self, inputs):
    z_mean, z_log_var = inputs
    epsilon = tf.random.normal(shape=tf.shape(z_mean))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class CVAE(keras.Model):
  def __init__(self, input_dim, cond_dim = 1, latent_dim = 2, beta = 1, **kwargs):
    super(CVAE, self).__init__(**kwargs)
    self.beta = beta

    #encoder
    x_input = keras.Input(shape=(input_dim,))
    cond_input = keras.Input(shape=(cond_dim,))
    x = layers.Concatenate()([x_input, cond_input])
    x = layers.Dense(32, activation="relu")(x)
    x = layers.Dense(16, activation="relu")(x)
    x = layers.Dense(8, activation="relu")(x)
    z_mean = layers.Dense(latent_dim)(x)
    z_log_var = layers.Dense(latent_dim)(x)

    z = Sampling()([z_mean, z_log_var])
    self.encoder = keras.Model([x_input, cond_input], [z_mean, z_log_var, z], name="encoder")

    #decoder
    latent_inputs = keras.Input(shape=(latent_dim,))
    cond_inputs_dec = keras.Input(shape=(cond_dim,))
    x = layers.Concatenate()([latent_inputs, cond_inputs_dec])
    x = layers.Dense(8, activation="relu")(x)
    x = layers.Dense(16, activation="relu")(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(input_dim, activation="linear")(x)

    self.decoder = keras.Model([latent_inputs, cond_inputs_dec], outputs, name="decoder")

  def train_step(self, data):
    (x, cond), y = data


    with tf.GradientTape() as tape:
      z_mean, z_log_var, z = self.encoder([x, cond], training = True)
      reconstruction = self.decoder([z, cond], training = True)

      reconstruction_loss = tf.reduce_mean(tf.square(x - reconstruction))
      kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
      total_loss = reconstruction_loss + self.beta * kl_loss

    grads = tape.gradient(total_loss, self.trainable_weights)
    self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

    return {"loss": total_loss, "recon_loss": reconstruction_loss, "kl_loss": kl_loss}

  def call(self, x, cond=None):
    # supporta sia call([x, cond]) che call(x, cond)
    if cond is None:
        # caso in cui arriva [x, cond] come lista
        x, cond = x
    z_mean, _, z = self.encoder([x, cond])
    return self.decoder([z, cond])


Scenario di danno DS1, 1 mese di dati

In [ ]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_20[features])
  T_val = scaler_T.transform(df_DS1_20[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_20)







df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_40[features])
  T_val = scaler_T.transform(df_DS1_40[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40,  "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_40)






df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_60[features])
  T_val = scaler_T.transform(df_DS1_60[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60,  "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_60)







df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_80[features])
  T_val = scaler_T.transform(df_DS1_80[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80,  "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_80)








df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_100[features])
  T_val = scaler_T.transform(df_DS1_100[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100,  "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS1, 2 mesi di dati

In [ ]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_20[features])
  T_val = scaler_T.transform(df_DS1_20[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_20)







df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_40[features])
  T_val = scaler_T.transform(df_DS1_40[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40,  "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_40)






df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_60[features])
  T_val = scaler_T.transform(df_DS1_60[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60,  "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_60)







df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_80[features])
  T_val = scaler_T.transform(df_DS1_80[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80,  "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_80)








df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_100[features])
  T_val = scaler_T.transform(df_DS1_100[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100,  "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS1, 4 mesi di dati

In [ ]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_20[features])
  T_val = scaler_T.transform(df_DS1_20[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_20)







df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_40[features])
  T_val = scaler_T.transform(df_DS1_40[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40,  "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_40)






df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_60[features])
  T_val = scaler_T.transform(df_DS1_60[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60,  "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_60)







df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_80[features])
  T_val = scaler_T.transform(df_DS1_80[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80,  "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_80)








df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_100[features])
  T_val = scaler_T.transform(df_DS1_100[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100,  "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS1, 6 mesi di dati

In [ ]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_20[features])
  T_val = scaler_T.transform(df_DS1_20[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_20)







df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_40[features])
  T_val = scaler_T.transform(df_DS1_40[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40,  "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_40)






df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_60[features])
  T_val = scaler_T.transform(df_DS1_60[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60,  "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_60)







df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_80[features])
  T_val = scaler_T.transform(df_DS1_80[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80,  "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_80)








df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_100[features])
  T_val = scaler_T.transform(df_DS1_100[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100,  "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS2, 1 mese di dati

In [ ]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_20[features])
  T_val = scaler_T.transform(df_DS2_20[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_20)







df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_40[features])
  T_val = scaler_T.transform(df_DS2_40[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40,  "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_40)






df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_60[features])
  T_val = scaler_T.transform(df_DS2_60[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60,  "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_60)







df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_80[features])
  T_val = scaler_T.transform(df_DS2_80[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80,  "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_80)








df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_100[features])
  T_val = scaler_T.transform(df_DS2_100[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100,  "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS2, 2 mesi di dati

In [ ]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_20[features])
  T_val = scaler_T.transform(df_DS2_20[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_20)







df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_40[features])
  T_val = scaler_T.transform(df_DS2_40[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40,  "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_40)






df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_60[features])
  T_val = scaler_T.transform(df_DS2_60[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60,  "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_60)







df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_80[features])
  T_val = scaler_T.transform(df_DS2_80[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80,  "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_80)








df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_100[features])
  T_val = scaler_T.transform(df_DS2_100[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100,  "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS2, 4 mesi di dati

In [ ]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_20[features])
  T_val = scaler_T.transform(df_DS2_20[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_20)







df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_40[features])
  T_val = scaler_T.transform(df_DS2_40[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40,  "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_40)






df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_60[features])
  T_val = scaler_T.transform(df_DS2_60[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60,  "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_60)







df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_80[features])
  T_val = scaler_T.transform(df_DS2_80[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80,  "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_80)








df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_100[features])
  T_val = scaler_T.transform(df_DS2_100[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100,  "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS2, 6 mesi di dati

In [ ]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_20[features])
  T_val = scaler_T.transform(df_DS2_20[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_20)







df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_40[features])
  T_val = scaler_T.transform(df_DS2_40[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40,  "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_40)






df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_60[features])
  T_val = scaler_T.transform(df_DS2_60[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60,  "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_60)







df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_80[features])
  T_val = scaler_T.transform(df_DS2_80[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80,  "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_80)








df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_100[features])
  T_val = scaler_T.transform(df_DS2_100[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100,  "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS3, 1 mese di dati

In [ ]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_20[features])
  T_val = scaler_T.transform(df_DS3_20[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_20)







df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_40[features])
  T_val = scaler_T.transform(df_DS3_40[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40,  "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_40)






df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_60[features])
  T_val = scaler_T.transform(df_DS3_60[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60,  "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_60)







df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_80[features])
  T_val = scaler_T.transform(df_DS3_80[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80,  "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_80)








df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_100[features])
  T_val = scaler_T.transform(df_DS3_100[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100,  "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS3, 2 mesi di dati

In [ ]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_20[features])
  T_val = scaler_T.transform(df_DS3_20[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_20)







df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_40[features])
  T_val = scaler_T.transform(df_DS3_40[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40,  "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_40)






df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_60[features])
  T_val = scaler_T.transform(df_DS3_60[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60,  "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_60)







df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_80[features])
  T_val = scaler_T.transform(df_DS3_80[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80,  "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_80)








df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_100[features])
  T_val = scaler_T.transform(df_DS3_100[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100,  "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS3, 4 mesi di dati

In [ ]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_20[features])
  T_val = scaler_T.transform(df_DS3_20[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_20)







df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_40[features])
  T_val = scaler_T.transform(df_DS3_40[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40,  "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_40)






df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_60[features])
  T_val = scaler_T.transform(df_DS3_60[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60,  "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_60)







df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_80[features])
  T_val = scaler_T.transform(df_DS3_80[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80,  "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_80)








df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_100[features])
  T_val = scaler_T.transform(df_DS3_100[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100,  "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS3, 6 mesi di dati

In [ ]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_20[features])
  T_val = scaler_T.transform(df_DS3_20[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_20)







df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_40[features])
  T_val = scaler_T.transform(df_DS3_40[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40,  "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_40)






df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_60[features])
  T_val = scaler_T.transform(df_DS3_60[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60,  "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_60)







df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_80[features])
  T_val = scaler_T.transform(df_DS3_80[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80,  "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_80)








df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_100[features])
  T_val = scaler_T.transform(df_DS3_100[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100,  "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS4, 1 mese di dati

In [ ]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_20[features])
  T_val = scaler_T.transform(df_DS4_20[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_20)







df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_40[features])
  T_val = scaler_T.transform(df_DS4_40[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40,  "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_40)






df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_60[features])
  T_val = scaler_T.transform(df_DS4_60[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60,  "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_60)







df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_80[features])
  T_val = scaler_T.transform(df_DS4_80[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80,  "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_80)








df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_100[features])
  T_val = scaler_T.transform(df_DS4_100[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100,  "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS4, 2 mesi di dati

In [ ]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_20[features])
  T_val = scaler_T.transform(df_DS4_20[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_20)







df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_40[features])
  T_val = scaler_T.transform(df_DS4_40[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40,  "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_40)






df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_60[features])
  T_val = scaler_T.transform(df_DS4_60[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60,  "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_60)







df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_80[features])
  T_val = scaler_T.transform(df_DS4_80[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80,  "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_80)








df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_100[features])
  T_val = scaler_T.transform(df_DS4_100[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100,  "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS4, 4 mesi di dati

In [ ]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_20[features])
  T_val = scaler_T.transform(df_DS4_20[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_20)







df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_40[features])
  T_val = scaler_T.transform(df_DS4_40[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40,  "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_40)






df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_60[features])
  T_val = scaler_T.transform(df_DS4_60[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60,  "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_60)







df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_80[features])
  T_val = scaler_T.transform(df_DS4_80[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80,  "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_80)








df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_100[features])
  T_val = scaler_T.transform(df_DS4_100[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100,  "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS4, 6 mesi di dati

In [ ]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_20[features])
  T_val = scaler_T.transform(df_DS4_20[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_20)







df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_40[features])
  T_val = scaler_T.transform(df_DS4_40[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40,  "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_40)






df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_60[features])
  T_val = scaler_T.transform(df_DS4_60[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60,  "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_60)







df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_80[features])
  T_val = scaler_T.transform(df_DS4_80[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80,  "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_80)








df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_100[features])
  T_val = scaler_T.transform(df_DS4_100[cond])

  cvae = CVAE(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae(dummy_x, dummy_cond)

  cvae.compile(optimizer=keras.optimizers.Adam())


  cvae.load_weights(f"/content/drive/MyDrive/CVAE_beta_1_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100,  "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

# CVAE beta 0.01

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="keras")

df_temp = pd.read_csv("/content/data_temperature.csv", encoding = "utf-8")
df_temp["timestamp"] = pd.to_datetime(df_temp["timestamp"])
df_temp["timestamp"] = df_temp["timestamp"].dt.tz_localize(None)
df_temp = df_temp.sort_values("timestamp")

end = "2025-01-01"
new_df = df_temp[(df_temp["timestamp"] < end)]

class Sampling(Layer):
  def call(self, inputs):
    z_mean, z_log_var = inputs
    epsilon = tf.random.normal(shape=tf.shape(z_mean))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class CVAE_beta001(keras.Model):
  def __init__(self, input_dim, cond_dim = 1, latent_dim = 2, beta = 0.01, **kwargs):
    super(CVAE_beta001, self).__init__(**kwargs)
    self.beta = beta

    #encoder
    x_input = keras.Input(shape=(input_dim,))
    cond_input = keras.Input(shape=(cond_dim,))
    x = layers.Concatenate()([x_input, cond_input])
    x = layers.Dense(32, activation="relu")(x)
    x = layers.Dense(16, activation="relu")(x)
    x = layers.Dense(8, activation="relu")(x)
    z_mean = layers.Dense(latent_dim)(x)
    z_log_var = layers.Dense(latent_dim)(x)

    z = Sampling()([z_mean, z_log_var])
    self.encoder = keras.Model([x_input, cond_input], [z_mean, z_log_var, z], name="encoder")

    #decoder
    latent_inputs = keras.Input(shape=(latent_dim,))
    cond_inputs_dec = keras.Input(shape=(cond_dim,))
    x = layers.Concatenate()([latent_inputs, cond_inputs_dec])
    x = layers.Dense(8, activation="relu")(x)
    x = layers.Dense(16, activation="relu")(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(input_dim, activation="linear")(x)

    self.decoder = keras.Model([latent_inputs, cond_inputs_dec], outputs, name="decoder")

  def train_step(self, data):
    (x, cond), y = data


    with tf.GradientTape() as tape:
      z_mean, z_log_var, z = self.encoder([x, cond], training = True)
      reconstruction = self.decoder([z, cond], training = True)

      reconstruction_loss = tf.reduce_mean(tf.square(x - reconstruction))
      kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
      total_loss = reconstruction_loss + self.beta * kl_loss

    grads = tape.gradient(total_loss, self.trainable_weights)
    self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

    return {"loss": total_loss, "recon_loss": reconstruction_loss, "kl_loss": kl_loss}

  def call(self, x, cond=None):
    # supporta sia call([x, cond]) che call(x, cond)
    if cond is None:
        # caso in cui arriva [x, cond] come lista
        x, cond = x
    z_mean, _, z = self.encoder([x, cond])
    return self.decoder([z, cond])


Scenario di danno DS1, 1 mese di dati

In [ ]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_20[features])
  T_val = scaler_T.transform(df_DS1_20[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_20)








df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_40[features])
  T_val = scaler_T.transform(df_DS1_40[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_40)





df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_60[features])
  T_val = scaler_T.transform(df_DS1_60[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_60)









df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_80[features])
  T_val = scaler_T.transform(df_DS1_80[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_80)








df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_100[features])
  T_val = scaler_T.transform(df_DS1_100[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS1, 2 mesi di dati

In [ ]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_20[features])
  T_val = scaler_T.transform(df_DS1_20[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_20)








df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_40[features])
  T_val = scaler_T.transform(df_DS1_40[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_40)





df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_60[features])
  T_val = scaler_T.transform(df_DS1_60[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_60)









df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_80[features])
  T_val = scaler_T.transform(df_DS1_80[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_80)








df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_100[features])
  T_val = scaler_T.transform(df_DS1_100[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS1, 4 mesi di dati

In [ ]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_20[features])
  T_val = scaler_T.transform(df_DS1_20[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_20)








df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_40[features])
  T_val = scaler_T.transform(df_DS1_40[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_40)





df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_60[features])
  T_val = scaler_T.transform(df_DS1_60[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_60)









df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_80[features])
  T_val = scaler_T.transform(df_DS1_80[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_80)








df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_100[features])
  T_val = scaler_T.transform(df_DS1_100[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS1, 6 mesi di dati

In [ ]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_20[features])
  T_val = scaler_T.transform(df_DS1_20[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_20)








df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_40[features])
  T_val = scaler_T.transform(df_DS1_40[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_40)





df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_60[features])
  T_val = scaler_T.transform(df_DS1_60[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_60)









df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_80[features])
  T_val = scaler_T.transform(df_DS1_80[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_80)








df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS1_100[features])
  T_val = scaler_T.transform(df_DS1_100[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS2, 1 mese di dati

In [ ]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_20[features])
  T_val = scaler_T.transform(df_DS2_20[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_20)








df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_40[features])
  T_val = scaler_T.transform(df_DS2_40[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_40)





df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_60[features])
  T_val = scaler_T.transform(df_DS2_60[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_60)









df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_80[features])
  T_val = scaler_T.transform(df_DS2_80[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_80)








df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_100[features])
  T_val = scaler_T.transform(df_DS2_100[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS2, 2 mesi di dati

In [ ]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_20[features])
  T_val = scaler_T.transform(df_DS2_20[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_20)








df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_40[features])
  T_val = scaler_T.transform(df_DS2_40[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_40)





df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_60[features])
  T_val = scaler_T.transform(df_DS2_60[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_60)









df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_80[features])
  T_val = scaler_T.transform(df_DS2_80[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_80)








df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_100[features])
  T_val = scaler_T.transform(df_DS2_100[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS2, 4 mesi di dati

In [ ]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_20[features])
  T_val = scaler_T.transform(df_DS2_20[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_20)








df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_40[features])
  T_val = scaler_T.transform(df_DS2_40[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_40)





df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_60[features])
  T_val = scaler_T.transform(df_DS2_60[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_60)









df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_80[features])
  T_val = scaler_T.transform(df_DS2_80[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_80)








df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_100[features])
  T_val = scaler_T.transform(df_DS2_100[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS2, 6 mesi di dati

In [ ]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_20[features])
  T_val = scaler_T.transform(df_DS2_20[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_20)








df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_40[features])
  T_val = scaler_T.transform(df_DS2_40[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_40)





df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_60[features])
  T_val = scaler_T.transform(df_DS2_60[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_60)









df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_80[features])
  T_val = scaler_T.transform(df_DS2_80[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_80)








df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS2_100[features])
  T_val = scaler_T.transform(df_DS2_100[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS3, 1 mese di dati

In [ ]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_20[features])
  T_val = scaler_T.transform(df_DS3_20[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_20)








df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_40[features])
  T_val = scaler_T.transform(df_DS3_40[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_40)





df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_60[features])
  T_val = scaler_T.transform(df_DS3_60[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_60)









df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_80[features])
  T_val = scaler_T.transform(df_DS3_80[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_80)








df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_100[features])
  T_val = scaler_T.transform(df_DS3_100[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS3, 2 mesi di dati

In [ ]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_20[features])
  T_val = scaler_T.transform(df_DS3_20[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_20)








df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_40[features])
  T_val = scaler_T.transform(df_DS3_40[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_40)





df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_60[features])
  T_val = scaler_T.transform(df_DS3_60[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_60)









df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_80[features])
  T_val = scaler_T.transform(df_DS3_80[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_80)








df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_100[features])
  T_val = scaler_T.transform(df_DS3_100[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS3, 4 mesi di dati

In [ ]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_20[features])
  T_val = scaler_T.transform(df_DS3_20[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_20)








df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_40[features])
  T_val = scaler_T.transform(df_DS3_40[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_40)





df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_60[features])
  T_val = scaler_T.transform(df_DS3_60[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_60)









df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_80[features])
  T_val = scaler_T.transform(df_DS3_80[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_80)








df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_100[features])
  T_val = scaler_T.transform(df_DS3_100[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS3, 6 mesi di dati

In [ ]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_20[features])
  T_val = scaler_T.transform(df_DS3_20[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_20)








df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_40[features])
  T_val = scaler_T.transform(df_DS3_40[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_40)





df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_60[features])
  T_val = scaler_T.transform(df_DS3_60[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_60)









df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_80[features])
  T_val = scaler_T.transform(df_DS3_80[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_80)








df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS3_100[features])
  T_val = scaler_T.transform(df_DS3_100[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS4, 1 mesi di dati

In [ ]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_20[features])
  T_val = scaler_T.transform(df_DS4_20[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_20)








df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_40[features])
  T_val = scaler_T.transform(df_DS4_40[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_40)





df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_60[features])
  T_val = scaler_T.transform(df_DS4_60[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_60)









df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_80[features])
  T_val = scaler_T.transform(df_DS4_80[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_80)








df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_100[features])
  T_val = scaler_T.transform(df_DS4_100[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS4, 2 mesi di dati

In [ ]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_20[features])
  T_val = scaler_T.transform(df_DS4_20[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_20)








df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_40[features])
  T_val = scaler_T.transform(df_DS4_40[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_40)





df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_60[features])
  T_val = scaler_T.transform(df_DS4_60[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_60)









df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_80[features])
  T_val = scaler_T.transform(df_DS4_80[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_80)








df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_100[features])
  T_val = scaler_T.transform(df_DS4_100[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS4, 4 mesi di dati

In [ ]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_20[features])
  T_val = scaler_T.transform(df_DS4_20[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_20)








df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_40[features])
  T_val = scaler_T.transform(df_DS4_40[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_40)





df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_60[features])
  T_val = scaler_T.transform(df_DS4_60[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_60)









df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_80[features])
  T_val = scaler_T.transform(df_DS4_80[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_80)








df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_100[features])
  T_val = scaler_T.transform(df_DS4_100[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS4, 6 mesi di dati

In [ ]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_20[features])
  T_val = scaler_T.transform(df_DS4_20[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_20)








df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_40[features])
  T_val = scaler_T.transform(df_DS4_40[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_40)





df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_60[features])
  T_val = scaler_T.transform(df_DS4_60[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_60)









df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_80[features])
  T_val = scaler_T.transform(df_DS4_80[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_80)








df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
cond = ["temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()
  scaler_T = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  T_train = scaler_T.fit_transform(train_data[cond])

  X_val = scaler_X.transform(df_DS4_100[features])
  T_val = scaler_T.transform(df_DS4_100[cond])

  cvae_beta001 = CVAE_beta001(input_dim=X_train.shape[1], cond_dim=T_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  dummy_cond = tf.zeros((1, T_train.shape[1]))
  _ = cvae_beta001(dummy_x, dummy_cond)

  cvae_beta001.compile(optimizer=keras.optimizers.Adam())


  cvae_beta001.load_weights(f"/content/drive/MyDrive/CVAE_beta_001_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = cvae_beta001.predict([X_train, T_train], verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = cvae_beta001.predict([X_val, T_val], verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

# VAE senza temperatura

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="keras")

df_temp = pd.read_csv("/content/data_temperature.csv", encoding = "utf-8")
df_temp["timestamp"] = pd.to_datetime(df_temp["timestamp"])
df_temp["timestamp"] = df_temp["timestamp"].dt.tz_localize(None)
df_temp = df_temp.sort_values("timestamp")

end = "2025-01-01"
new_df = df_temp[(df_temp["timestamp"] < end)]



class Sampling(Layer):
  def call(self, inputs):
    z_mean, z_log_var = inputs
    epsilon = tf.random.normal(shape=tf.shape(z_mean))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon



class VAE(keras.Model):
  def __init__(self, input_dim, latent_dim = 2, beta = 0.01, **kwargs):
    super(VAE, self).__init__(**kwargs)
    self.beta = beta

    #encoder
    x_input = keras.Input(shape=(input_dim,))
    x = layers.Dense(48, activation="relu")(x_input)
    x = layers.Dense(24, activation="relu")(x)
    x = layers.Dense(12, activation="relu")(x)
    z_mean = layers.Dense(latent_dim)(x)
    z_log_var = layers.Dense(latent_dim)(x)

    z = Sampling()([z_mean, z_log_var])
    self.encoder = keras.Model(x_input, [z_mean, z_log_var, z])

    #decoder
    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.Dense(12, activation="relu")(latent_inputs)
    x = layers.Dense(24, activation="relu")(x)
    x = layers.Dense(48, activation="relu")(x)
    outputs = layers.Dense(input_dim, activation="linear")(x)

    self.decoder = keras.Model(latent_inputs, outputs)

  def train_step(self, data):
    x, y = data

    with tf.GradientTape() as tape:
      z_mean, z_log_var, z = self.encoder(x, training = True)
      reconstruction = self.decoder(z, training = True)

      reconstruction_loss = tf.reduce_mean(tf.square(x - reconstruction))
      kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
      total_loss = reconstruction_loss + self.beta * kl_loss

    grads = tape.gradient(total_loss, self.trainable_weights)
    self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

    return {"loss": total_loss, "recon_loss": reconstruction_loss, "kl_loss": kl_loss}

  def call(self, inputs):
    z_mean, _,  z = self.encoder(inputs)
    return self.decoder(z)

Scenario di danno DS1, 1 mese di dati

In [ ]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_20[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_20)







df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_40[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_40)






df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_60[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_60)









df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_80[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_80)







df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_100[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS1, 2 mesi di dati

In [ ]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_20[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_20)







df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_40[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_40)






df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_60[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_60)






df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_80[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_80)





df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_100[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS1, 4 mesi di dati

In [ ]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_20[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_20)







df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_40[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_40)






df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_60[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_60)









df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_80[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_80)





df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_100[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS1, 6 mesi di dati

In [ ]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_20[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_20)







df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_40[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_40)






df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_60[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_60)







df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_80[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_80)





df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS1_100[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS2, 1 mese di dati

In [ ]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_20[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_20)







df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_40[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_40)






df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_60[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_60)






df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_80[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_80)






df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_100[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS2, 2 mesi di dati

In [ ]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_20[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_20)







df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_40[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_40)






df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_60[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_60)








df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_80[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_80)





df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_100[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS2, 4 mesi di dati

In [ ]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_20[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_20)







df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_40[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_40)






df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_60[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_60)







df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_80[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_80)






df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_100[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS2, 6 mesi di dati

In [ ]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_20[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_20)







df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_40[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_40)






df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_60[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_60)






df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_80[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_80)





df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS2_100[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS3, 1 mese di dati

In [ ]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_20[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_20)







df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_40[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_40)






df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_60[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_60)





df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_80[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_80)






df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_100[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS3, 2 mesi di dati

In [ ]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_20[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_20)







df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_40[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_40)






df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_60[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_60)





df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_80[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_80)






df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_100[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS3, 4 mesi di dati

In [ ]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_20[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_20)







df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_40[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_40)






df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_60[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_60)





df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_80[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_80)






df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_100[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS3, 6 mesi di dati

In [ ]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_20[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_20)







df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_40[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_40)






df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_60[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_60)





df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_80[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_80)






df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS3_100[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS4, 1 mese di dati

In [ ]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_20[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_20)







df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_40[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_40)






df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_60[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_60)





df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_80[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_80)






df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_100[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS4, 2 mesi di dati

In [ ]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_20[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_20)







df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_40[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_40)






df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_60[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_60)





df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_80[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_80)






df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_100[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS4, 4 mesi di dati

In [ ]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_20[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_20)







df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_40[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_40)






df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_60[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_60)





df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_80[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_80)






df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_100[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS4, 6 mesi di dati

In [ ]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_20[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_20)







df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_40[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_40)






df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_60[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_60)





df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_80[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_80)






df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])

  X_val = scaler_X.transform(df_DS4_100[features])

  vae_no_temp = VAE(input_dim=X_train.shape[1], latent_dim=2)

  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_no_temp(dummy_x)

  vae_no_temp.compile(optimizer=keras.optimizers.Adam())


  vae_no_temp.load_weights(f"/content/drive/MyDrive/VAE_no_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_no_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_no_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(X_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

# VAE con temperatura

In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="keras")

df_temp = pd.read_csv("/content/data_temperature.csv", encoding = "utf-8")
df_temp["timestamp"] = pd.to_datetime(df_temp["timestamp"])
df_temp["timestamp"] = df_temp["timestamp"].dt.tz_localize(None)
df_temp = df_temp.sort_values("timestamp")

end = "2025-01-01"
new_df = df_temp[(df_temp["timestamp"] < end)]



class Sampling(Layer):
  def call(self, inputs):
    z_mean, z_log_var = inputs
    epsilon = tf.random.normal(shape=tf.shape(z_mean))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon



class VAE_temp(keras.Model):
  def __init__(self, input_dim, output_dim, latent_dim = 2, beta = 0.01, **kwargs):
    super(VAE_temp, self).__init__(**kwargs)
    self.beta = beta

    #encoder
    x_input = keras.Input(shape=(input_dim,))
    x = layers.Dense(32, activation="relu")(x_input)
    x = layers.Dense(16, activation="relu")(x)
    x = layers.Dense(8, activation="relu")(x)
    z_mean = layers.Dense(latent_dim)(x)
    z_log_var = layers.Dense(latent_dim)(x)

    z = Sampling()([z_mean, z_log_var])
    self.encoder = keras.Model(x_input, [z_mean, z_log_var, z])

    #decoder
    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.Dense(8, activation="relu")(latent_inputs)
    x = layers.Dense(16, activation="relu")(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(output_dim, activation="linear")(x)

    self.decoder = keras.Model(latent_inputs, outputs)

  def train_step(self, data):
    x, y = data

    with tf.GradientTape() as tape:
      z_mean, z_log_var, z = self.encoder(x, training = True)
      reconstruction = self.decoder(z, training = True)

      reconstruction_loss = tf.reduce_mean(tf.square(x[:, :6] - reconstruction))
      kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
      total_loss = reconstruction_loss + self.beta * kl_loss

    grads = tape.gradient(total_loss, self.trainable_weights)
    self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

    return {"loss": total_loss, "recon_loss": reconstruction_loss, "kl_loss": kl_loss}

  def call(self, inputs):
    z_mean, _,  z = self.encoder(inputs)
    return self.decoder(z)


Scenario di danno DS1, 1 mese di dati

In [4]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_20[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_20)








df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_40[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_40)









df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_60[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_60)







df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_80[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_80)






df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_100[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS1, 2 mesi di dati

In [5]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_20[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_20)








df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_40[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_40)









df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_60[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_60)







df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_80[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_80)






df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_100[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS1, 4 mesi di dati

In [6]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_20[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_20)








df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_40[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_40)









df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_60[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_60)







df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_80[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_80)






df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_100[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS1, 6 mesi di dati

In [7]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_20[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_20)








df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_40[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_40)









df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_60[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_60)







df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_80[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_80)






df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_100[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS2, 1 mese di dati

In [8]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_20[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_20)








df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_40[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_40)









df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_60[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_60)







df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_80[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_80)






df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_100[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS2, 2 mesi di dati

In [9]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_20[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_20)








df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_40[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_40)









df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_60[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_60)







df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_80[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_80)






df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_100[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS2, 4 mesi di dati

In [10]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_20[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_20)








df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_40[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_40)









df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_60[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_60)







df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_80[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_80)






df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_100[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS2, 6 mesi di dati

In [11]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_20[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_20)








df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_40[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_40)









df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_60[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_60)







df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_80[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_80)






df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_100[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS3, 1 mese di dati

In [12]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_20[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_20)








df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_40[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_40)









df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_60[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_60)







df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_80[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_80)






df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_100[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS3, 2 mesi di dati

In [13]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_20[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_20)








df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_40[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_40)









df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_60[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_60)







df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_80[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_80)






df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_100[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS3, 4 mesi di dati

In [14]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_20[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_20)








df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_40[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_40)









df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_60[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_60)







df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_80[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_80)






df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_100[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS3, 6 mesi di dati

In [15]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_20[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_20)








df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_40[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_40)









df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_60[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_60)







df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_80[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_80)






df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_100[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS4, 1 mese di dati

In [16]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_20[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_20)








df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_40[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_40)









df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_60[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_60)







df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_80[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_80)






df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_100[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_1month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS4, 2 mesi di dati

In [17]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_20[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_20)








df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_40[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_40)









df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_60[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_60)







df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_80[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_80)






df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_100[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_2month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS4, 4 mesi di dati

In [18]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_20[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_20)








df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_40[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_40)









df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_60[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_60)







df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_80[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_80)






df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_100[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_4month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS4, 6 mesi di dati

In [19]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_20[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_20)








df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_40[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_40)









df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_60[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_60)







df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_80[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_80)






df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = new_df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = new_df[(new_df["timestamp"] >= train_start) & (new_df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_100[features])

  Y_train = X_train[:, :6]
  Y_val = X_val[:, :6]

  vae_temp = VAE_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1], latent_dim=2)


  # Inizializzare il modello con tensori dummy
  dummy_x = tf.zeros((1, X_train.shape[1]))
  _ = vae_temp(dummy_x)

  vae_temp.compile(optimizer=keras.optimizers.Adam())


  vae_temp.load_weights(f"/content/drive/MyDrive/VAE_temp_6month_{m}.weights.h5")


  #calcolo della soglia su dati normali
  X_pred_train = vae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(Y_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  X_pred = vae_temp.predict(X_val, verbose=0)
  mse = np.mean(np.square(Y_val - X_pred), axis=1)

  anomalies = mse > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months),  "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     